In [1]:
!pip install -q google-genai

In [2]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
from google import genai

client = genai.Client()

modelo = "gemini-2.0-flash"

In [4]:
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

In [6]:
def call_agent(agent: Agent, message_text: str) -> str:

    session_service = InMemorySessionService()

    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""

    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
def agente_consultor_legislativo(atividade, tipo_de_tributacao, estado):

  consultor = Agent(
      name="agente_consultor_legislativo",
      model="gemini-2.0-flash",
      instruction="""
      Você é um especialista em legislação tributária brasileira.
      Sua função é buscar atualizações recentes sobre a reforma tributária, especialmente a Emenda Constitucional 132/2023, e apresentar
      resumos objetivos e técnicos sobre as alterações legais relacionadas ao ramo de atividade, tipo de tributação e Estado do usuário.
      Utilize linguagem clara, técnica e focada em profissionais da área contábil e
      fiscal e empresários. Evite opiniões pessoais. Sempre cite a fonte e a data da última atualização.


      """,
      description="Agente que consulta a legislação vigente sobre a Reforma Tributária.",
      tools=[google_search]
  )

  entrada_do_agente_consultor_legislativo = f"Atividade Principal da Empresa: {atividade}\n Tipo de Tributação: {tipo_de_tributacao}\n Região: {estado}"

  atualizacoes = call_agent(consultor, entrada_do_agente_consultor_legislativo)

  return atualizacoes

In [9]:
def agente_analisador_tributario(atividade, tipo_de_tributacao, estado, atualizacoes):

  analisador = Agent(
      name="agente_analisador_tributario",
      model="gemini-2.0-flash",
      instruction="""
      Você é um analista tributário com profundo conhecimento da legislação brasileira (EC 132/2023) e da reforma tributária.
      Sua função é interpretar o conteúdo legal e identificar como as mudanças afetam setores econômicos,
      tipos de tributação (MEI, Simples, Lucro Real, Presumido) e diferentes regiões. Com base nas atualizações encontradas pelo
      agente consultor legislativo, você irá analisar o caso específico do usuário, que irá informar seu setor econômico, tipo de empresa e região.
      Use comparações antes/depois e destaque os pontos mais relevantes de forma analítica em um breve relatório de análise.
      Seja claro, preciso e evite linguagem excessivamente técnica.
      """,
      description="Agente que analisa os impactos da reforma tributária.",
  )

  entrada_do_agente_analisador_tributario = f"Atividade Principal da Empresa: {atividade}\n Tipo de Tributação: {tipo_de_tributacao}\n Região: {estado}\n Análise: {atualizacoes}"

  analise = call_agent(analisador, entrada_do_agente_analisador_tributario)

  return analise

In [10]:
def agente_contador(atividade, tipo_de_tributacao, estado, analise):

  contador = Agent(
      name="agente_contador",
      model="gemini-2.0-flash",
      instruction="""
      Você é um contador experiente, especializado na reforma tributária. Com base na análise de caso gerada
      pelo agente analisador tributário, elabore um planejamento contábil completo, focado nas informações
      fornecidas pelo usuário (setor econômico, tipo de empresa e região). O plano contábil deve seguir as
      normas vigentes propostas pelo Conselho Federal de Contabilidade e Normas Brasileiras de Contabilidade.
      """,
      description="Agente que traça um plano contábil para o usuário."
  )

  entrada_do_agente_contador = f"Atividade Principal da Empresa: {atividade}\n Tipo de Tributação: {tipo_de_tributacao}\n Região: {estado}\n Análise: {analise}"

  plano_contabil = call_agent(contador, entrada_do_agente_contador)

  return plano_contabil

In [11]:
print(" Bem vindo(a) ao TributAI, seu assistente contábil virtual.\n ")


atividade = input("Por favor, digite o RAMO DE ATIVIDADE da sua empresa:")
tipo_de_tributacao = input("❓ Por favor, digite o TIPO DE TRIBUTAÇÃO atual da sua empresa: ")
estado = input("❓ Por favor, digite a REGIÃO que você se encontra: ")


if not atividade or not tipo_de_tributacao or not estado:
  print("Você esqueceu de digitar alguma informação!\n")

else:
    print(f"Maravilha! Vamos então iniciar a análise!.\n")

    atualizacoes = agente_consultor_legislativo(atividade, tipo_de_tributacao, estado)
    print("\n📝 Resultado da Consulta à Legislação:\n")
    display(to_markdown(atualizacoes))
    print("-" * 120)

    analise = agente_analisador_tributario(atividade, tipo_de_tributacao, estado, atualizacoes)
    print("\n📝 Resultado da Análise Tributária:\n")
    display(to_markdown(analise))
    print("-" * 120)

    plano_contabil = agente_contador(atividade, tipo_de_tributacao, estado, analise)
    print("\n📝 Aqui está seu Planejamento Contábil completo!\n")
    display(to_markdown(plano_contabil))
    print("-" * 120)


 Bem vindo(a) ao TributAI, seu assistente contábil virtual.
 


KeyboardInterrupt: Interrupted by user

In [12]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 114.0 MB/s eta 0:00:00


In [15]:
import gradio as gr

In [18]:
def interface_tributai(atividade, tipo_de_tributacao, estado):
    try:
        atualizacoes = agente_consultor_legislativo(atividade, tipo_de_tributacao, estado)
        analise = agente_analisador_tributario(atividade, tipo_de_tributacao, estado, atualizacoes)
        plano = agente_contador(atividade, tipo_de_tributacao, estado, analise)

        return (
            f"🧾 **Atualizações Legislativas:**\n\n{atualizacoes}\n\n"
            f"📊 **Análise Tributária:**\n\n{analise}\n\n"
            f"📋 **Plano Contábil Sugerido:**\n\n{plano}"
        )
    except Exception as e:
        return f"❌ Ocorreu um erro: {str(e)}"


custom_theme = gr.themes.Base().set(
    body_background_fill="#f9f9f9",      # fundo da página
    body_text_color="#333333",           # cor do texto
    block_background_fill="#f0f4f8",     # fundo dos componentes
    block_border_color="#e0e0e0",        # bordas suaves
)


interface = gr.Interface(
    fn=interface_tributai,
    inputs=[
        gr.Textbox(label="🧾 Atividade Principal da Empresa"),
        gr.Textbox(label="💼 Tipo de Tributação (MEI, Simples Nacional, Lucro Real, Lucro Presumido)"),
        gr.Textbox(label="🌎 Estado ou Região")
    ],
    outputs=gr.Markdown(label="📄 Resultado da Análise"),
    title="TributAI - Assistente Contábil especializado na Reforma Tributária",
    description="Informe os dados da sua empresa para receber uma análise completa e atualizada da reforma tributária (EC 132/2023).",
    theme=custom_theme
)


interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a1b0ba80a92104ba4a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
